In [1]:
%pip install nest_asyncio load_dotenv

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append('/Users/arthurj/Git_Repos/LightRAG')

In [3]:
import logging
import time
import json
import os
import pdfplumber
import nest_asyncio
from load_dotenv import load_dotenv
from dataclasses import asdict, dataclass
from lightrag.lightrag import LightRAG, QueryParam
from lightrag.llm import gpt_4o_mini_complete, gpt_4o_complete_batch


NameError: name 'Dict' is not defined

In [ ]:
nest_asyncio.apply()

In [ ]:
# enviroment setup

WORKING_DIR = "/Users/arthurj/Git_Repos/LightRAG/lightrag_expriments/notebooks/SnP_directory"
if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

dotenv_path = '/Users/arthurj/Git_Repos/LightRAG/lightrag_expriments/.env'
load_dotenv(dotenv_path)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_KEY')

In [ ]:
# model using 4o, text-embedding-3-small & moodys specific prompt to match banking inputs 

rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=gpt_4o_complete_batch
)

#### Ingestion

In [ ]:
#set up ingesiton method for SnP data

ratings_file_path = "/Users/arthurj/Library/CloudStorage/OneDrive-moodys.com/Desktop/LightRag_Data/Ratings"

def process_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):

        file_path = os.path.join(folder_path, filename)

        if os.path.isfile(file_path) and filename.endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as f:
                rag.insert(f.read())
                print(f"Processed: {filename}")

            break
    

process_files_in_folder(ratings_file_path)

In [ ]:
#print(rag.query("Given the rating data on Merck, please provide an overview of the companies outlook?", param=QueryParam(mode="hybrid")))


#### Visualization of Graph 

In [ ]:
import networkx as nx
from pyvis.network import Network

graph_path = os.path.join(WORKING_DIR, 'graph_chunk_entity_relation.graphml')
G = nx.read_graphml(graph_path)

net = Network(notebook=True, cdn_resources="in_line")
net.from_nx(G)
output_file = os.path.join(WORKING_DIR, 'knowledge_graph.html')
net.show(output_file)

print(f"Interactive knowledge graph saved to: {output_file}")


#### Query Testing

In [ ]:
query_file_path = "/Users/arthurj/Git_Repos/ma-cpg-genai-research/graphrag-playground/lightrag/notebooks/example_queries.json"
output_file = os.path.join(WORKING_DIR, "../sales_documents/output_response.txt")

In [ ]:
def write_to_file(output_text):
   with open(output_file, "a", encoding="utf-8") as file:
       file.write(output_text + "\n")

In [ ]:
def load_queries_from_json(json_path):
    with open(json_path, "r", encoding="utf-8") as file:
        queries_data = json.load(file)
    return queries_data

In [ ]:
def process_queries_from_json(json_path):
    queries_data = load_queries_from_json(json_path)
    
    for item in queries_data:  
        query_number = item["example_number"]
        query_text = item["query"]
        
        for mode in ["hybrid"]: #"naive", "local", "global"
            response = rag.query(query_text, param=QueryParam(mode=mode))
            output_text = (
                f"Example Number: {example_number}, Mode: {mode}\n"
                f"Query: {query_text}\n"
                f"Response: {response}\n"
                "------------------------------------\n"
            )
            write_to_file(output_text)
            print(f"Processed Example Number: {example_number}, Mode: {mode}")

In [ ]:
process_queries_from_json(json_file_path)

#### Evaluation 

In [ ]:
# Evalutation method 